# CS 773 Data Mining and Security

## Course Project - Analyzing Open University Learning Analytics Dataset 

### Problem Statement 

Based on the given data of students enrolled in various courses identify the student who is at-risk so that suitable interventions can be taken to help students succeed. 

***Part-4 : Data Modelling and Prediction***

In this part we will try to model the data and predict the student at risk.

In [3]:
#importing Libraries
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ML libraries
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm.libsvm import predict_proba
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve

In [4]:
# Reading Data
student_info = pd.read_csv('../data/studentInfo.csv')
student_assessment = pd.read_csv('../data/studentAssessment.csv')
student_registration = pd.read_csv('../data/studentRegistration.csv')
student_vle = pd.read_csv('../data/studentVle.csv')
assessments = pd.read_csv('../data/assessments.csv')
courses = pd.read_csv('../data/courses.csv')
vle = pd.read_csv('../data/vle.csv')

#check for the data
print('----- Rows -----')
print('Courses     : ',courses.shape[0])
print('Assessments : ',assessments.shape[0])
print('vle(Virtual Learning Environment) : ',vle.shape[0])
print('Student Info         : ',student_info.shape[0])
print('student Assessment   : ',student_assessment.shape[0])
print('student Registration : ',student_registration.shape[0])
print('student Vle          : ',student_vle.shape[0])

----- Rows -----
Courses     :  22
Assessments :  206
vle(Virtual Learning Environment) :  6364
Student Info         :  32593
student Assessment   :  173912
student Registration :  32593
student Vle          :  10655280


In [6]:
#Utilities
"""
Calculates sum based on the previous value
"""
def compute_weighted_sum(data, current_val, new_col):
    data[new_col] = data[current_val].cumsum()
    return data

In [7]:
"""
Get mean of the split data
"""
def split_mean(x):
    split_list = x.split('-')
    mean = (float(split_list[0])+float(split_list[1]))/2
    return mean

In [8]:
"""
Preprocessing Student_info
"""

def preprocess_student_info(data):
    data_pre = data.copy()
    data_pre['final_result'] = labelEncoder.fit_transform(data_pre['final_result'])
    #convert categorical columns 
    #----imd_band
    # Remove % first and replace '?' with 0 
    data_pre['imd_band'] =  data_pre['imd_band'].apply(lambda x : x.replace('%',''))
    data_pre['imd_band'] =  data_pre['imd_band'].apply(lambda x : '0' if x == '?' else x)
    #compute mean of the range and add them in seperate column
    data_pre['imd_band_mean'] = data_pre['imd_band'].apply(lambda x : 0 if x == '0' else split_mean(x))
    #----age_band
    # Replace '55<=' with 55 and other ranges with their mean 
    data_pre['age_band_mean'] =  data_pre['age_band'].apply(lambda x : 55 if x == '55<=' else split_mean(x))
    data_pre = data_pre[['code_module','code_presentation','id_student','gender','region','highest_education','num_of_prev_attempts','studied_credits','imd_band','imd_band_mean','age_band','age_band_mean','final_result']]
    return data_pre

In [9]:
#adding some more columns
assessments_stats = assessments.groupby(['code_module','code_presentation']).agg({'weight' : 'sum'})
assessments_stats.reset_index(level=[0,1], inplace=True)
assessments_stats.columns = ['code_module','code_presentation','total_marks']
assessments_stats = pd.merge(assessments, assessments_stats, how='left', on=['code_module','code_presentation'])
assessments_stats['percent_of_total'] = np.divide(assessments_stats['weight'],assessments_stats['total_marks'])*100
#assessments_stats.groupby(['code_module','code_presentation']).size()
assessments_stats = assessments_stats.groupby(['code_module','code_presentation']).apply(compute_weighted_sum, 'percent_of_total', 'weight_in_total')
student_assessments_all = pd.merge(student_assessment, assessments_stats, how = 'outer', on='id_assessment' )
student_assessments_all['score'] = np.where(student_assessments_all['score'] == '?', '0', student_assessments_all['score'])
student_assessments_all['score'] = pd.to_numeric(student_assessments_all['score'])
student_assessments_all['assessment_result'] = np.where(student_assessments_all['score'] >= 40, 1, 0)
student_assessments_all['score_weight'] = np.divide(student_assessments_all['score'],100)*student_assessments_all['percent_of_total']
student_assessments_all = student_assessments_all.dropna()
#student_assessments_all.id_student.nunique()
student_assessments_all = student_assessments_all.groupby(['code_module','code_presentation','id_student']).apply(compute_weighted_sum,'score_weight', 'score_in_total')
student_assessments_all = student_assessments_all[['code_module','code_presentation','id_student','id_assessment','assessment_type','date','date_submitted','is_banked',
                                                   'score','weight','percent_of_total','score_weight','weight_in_total','score_in_total','total_marks','assessment_result']]
#student_assessments_all[student_assessments_all['id_student'] == 63400]
student_assessments_all = student_assessments_all.dropna()
student_assessments_all[student_assessments_all['id_student'] == 63400]

,code_module,code_presentation,id_student,id_assessment,assessment_type,date,date_submitted,is_banked,score,weight,percent_of_total,score_weight,weight_in_total,score_in_total,total_marks,assessment_result
13,AAA,2013J,63400.0,1752,TMA,19,19.0,0.0,83.0,10.0,5.0,4.15,5.0,4.15,200.0,1
514,AAA,2013J,63400.0,1753,TMA,54,54.0,0.0,42.0,20.0,10.0,4.20,15.0,8.35,200.0,1
855,AAA,2013J,63400.0,1754,TMA,117,117.0,0.0,78.0,20.0,10.0,7.80,25.0,16.15,200.0,1
1045,AAA,2013J,63400.0,1755,TMA,166,166.0,0.0,76.0,20.0,10.0,7.60,35.0,23.75,200.0,1
1470,AAA,2013J,63400.0,1756,TMA,215,215.0,0.0,79.0,30.0,15.0,11.85,50.0,35.60,200.0,1
